In [ ]:
import sys; sys.path.append('..')
import numpy as np, importlib
import mesh, tri_mesh_viewer, fd_validation, parallelism

In [ ]:
#parallelism.set_max_num_tbb_threads(1)
#parallelism.set_gradient_assembly_num_threads(1)
#parallelism.set_hessian_assembly_num_threads(1)

np.random.seed(0)

m = mesh.Mesh('../../misc/examples/meshes/square_hole.off', embeddingDimension=3)

In [ ]:
#v = tri_mesh_viewer.TriMeshViewer(m, wireframe=True, width=1024, height=768)
#v.show()

In [ ]:
creases = [[40, 210], [210, 56], [56, 212], [212, 58], [58, 216], [216, 46]] # Diagonal chosen from the mesh
#creases = np.zeros((0, 2), dtype=np.int)

In [ ]:
import elastic_sheet, energy, tensors

#et = tensors.ElasticityTensor2D(200, 0.3)
#psi = energy.StVenantKirchhoffCBased(et)
psi = energy.NeoHookeanYoungPoisson(2, 200, 0.3)
es = elastic_sheet.ElasticSheet(m, psi, creaseEdges=creases)
es.hessianProjectionType = es.HPType.Off

In [ ]:
import elastic_solid
m2d = mesh.Mesh('../../misc/examples/meshes/square_hole.off', embeddingDimension=2)

In [ ]:
perturb2D = 1e-2 * np.random.normal(size=(m2d.numVertices(), 2))
perturb3D = np.pad(perturb2D, [(0, 0), (0, 1)])

In [ ]:
es.setDeformedPositions(es.getDeformedPositions() + perturb3D)

In [ ]:
importlib.reload(fd_validation)
fd_validation.gradConvergencePlot(es, customArgs={'updatedSource': False, 'etype': es.EnergyType.Membrane}) # non-updated source

In [ ]:
# Now we perturb out of plane and test the bending energy gradients
es.setVars(es.getVars() + np.random.normal(size=es.numVars()))
#es.updateSourceFrame()
deltaVar = np.random.normal(size=es.numVars())

In [ ]:
#es.updateSourceFrame()

In [ ]:
# es.setIdentityDeformation()
# es.setThetas(np.random.normal(size=len(es.getThetas())))
# 
# deltaVar = np.random.normal(size=es.numVars())
# #deltaVar[:es.thetaOffset()] = 0.0
# deltaVar[es.thetaOffset():] = 0.0
# deltaVar[:es.thetaOffset()] = np.pad(np.random.normal(size=(m.numVertices(), 2)), [(0, 0), (0, 1)]).ravel() # Perturb in plane only
# deltaVar[:] = 0.0
# deltaVar[1] = 1.0

In [ ]:
#es.setCreaseAngles(np.zeros_like(es.getCreaseAngles()))

In [ ]:
es.getCreaseAngles()

In [ ]:
importlib.reload(fd_validation)
#deltaVar[es.creaseAngleOffset():] = 0
fd_validation.gradConvergencePlot(es, customArgs={'updatedSource': False, 'etype': es.EnergyType.Bending}, perturb=deltaVar) # non-updated source

In [ ]:
gv = fd_validation.validateGrad(es, customArgs={'updatedSource': False, 'etype': es.EnergyType.Bending}, perturb=deltaVar)

In [ ]:
gv = fd_validation.validateGrad(es, customArgs={'updatedSource': True, 'etype': es.EnergyType.Bending}, perturb=deltaVar)

In [ ]:
gv

In [ ]:
importlib.reload(fd_validation)
fd_validation.gradConvergencePlot(es, customArgs={'updatedSource': False, 'etype': es.EnergyType.Bending}, perturb=deltaVar) # non-updated source

In [ ]:
es.updateSourceFrame()

In [ ]:
# Now we perturb out of plane and test the bending energy gradients
es.setIdentityDeformation()
es.setVars(es.getVars() + 1e-1 * np.random.normal(size=es.numVars()))
es.updateSourceFrame()
deltaVar = np.random.normal(size=es.numVars())

In [ ]:
perturb = np.random.normal(size=es.numVars())
#perturb[0:es.thetaOffset()] = 0.0

In [ ]:
currVars = es.getVars()

In [ ]:
es.updateSourceFrame()

In [ ]:
ca = {'updatedSource': False, 'etype': es.EnergyType.Bending}
r = fd_validation.validateHessian(es, perturb=perturb, customArgs=ca, fd_eps=1e-6)
print(r[0])
comp = np.array(r[1:])
print(comp)

In [ ]:
v = 0
print(comp[:, 3*v:(3*v+3)])

In [ ]:
es.updateSourceFrame()

In [ ]:
#perturb[0:es.thetaOffset()] = 0.0
#perturb[es.creaseAngleOffset():] = 0.0

In [ ]:
es.setCreaseAngles(np.zeros_like(es.getCreaseAngles()))

In [ ]:
#d_validation.hessConvergencePlot(es, customArgs={'updatedSource': False, 'etype': es.EnergyType.Bending}, perturb=perturb, indexInterval=[0, es.creaseAngleOffset()])
fd_validation.hessConvergencePlot(es, customArgs={'updatedSource': False, 'etype': es.EnergyType.Bending}, perturb=perturb)